In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.collections import LineCollection

import re
import sys
import os
import json

path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# sys.path.append("/home/kalmar/Mathematics/Software/Dionysus/build-x64/bindings/python/")
sys.path.append(os.path.join(path,"Dionysus-python3/build/bindings/python"))
import dionysus

%reload_ext autoreload
%autoreload 2

import TDA

In [4]:
prefix = 'sampled-2000'
prefix = 'test'
pattern = re.compile('pair[0-9]{4,}$')
files = sorted([x for x in os.listdir(os.path.join(os.getcwd(),prefix)) if pattern.match(x)])
print(len(files))
print(files[:4])

83
['pair0001', 'pair0002', 'pair0003', 'pair0004']


In [7]:
i = 49
print(files[i])
directory = directory = os.path.join(os.getcwd(),prefix,files[i])
with open(os.path.join(directory,"diagrams_knn")) as file:
    diagrams = json.load(file)

pair0054


In [8]:
for key, value in diagrams[0].items():
    for i, pairs in enumerate(value): 
        print("projection on",i,"axis", key)
        print(pairs)

projection on 0 axis X
[[1.4394171999239531, 1.6900722076183898]]
projection on 1 axis X
[[1.3322045080447715, 1.4329936227473654]]
projection on 2 axis X
[]
projection on 0 axis Y
[[1.4453953140918707, 1.5108128046752323]]
projection on 1 axis Y
[[1.5330953421250708, 1.5987876974384105]]
projection on 0 axis Y_inverted
[]
projection on 1 axis Y_inverted
[[3.484979449372683, 3.5383544880647717]]
projection on 0 axis X_inverted
[[1.8367670235969369, 3.7195559502459674]]
projection on 1 axis X_inverted
[[0.20401667572863813, 0.27635724999098343]]
projection on 2 axis X_inverted
[[0.19053334555839552, 0.3297692519279929]]


In [9]:
class dummy:
    
    def __init__(self, diagrams):
        self.diagrams_list = diagrams
        self.create_persistence_diagrams()
    
    def create_persistence_diagrams(self):
        self.persistence_diagrams = []
        for dictionary in self.diagrams_list:
            dic = {}
            for key, projection in dictionary.items():
                proj = []
                for diagram in projection:
                    p_diagram = dionysus.PersistenceDiagram(0)
                    for pair in diagram:
                        p_diagram.append(tuple(pair))
                    proj.append(p_diagram)
                dic[key] = proj
            self.persistence_diagrams.append(dic)

In [17]:
x = dummy(diagrams)
x.persistence_diagrams[23]["X"][0]

1.71405 2.01252

In [14]:
len(x.persistence_diagrams)

58

In [15]:
diagrams[23]["X"][0]

[[1.7140457170643943, 2.012523561029362]]

In [ ]:
# for each in files[3:]:
#     p=Pair(prefix,each)

In [49]:
class ComputedResults:
    
    def __init__(self, prefix='test', outlier_model='knn'):
    
        pattern = re.compile('pair[0-9]{4,}$')
        prefix_path = os.path.join(os.getcwd(),prefix)
        dir_list = sorted([x for x in os.listdir(prefix_path) if pattern.match(x)])
        self.pairs = []
        
        for directory in dir_list:
            pair_dir = os.path.join(os.getcwd(),prefix,directory)
            if outlier_model == 'knn':
                path_to_diagrams = os.path.join(pair_dir,"diagrams_knn")
            elif outlier_model == 'all':
                path_to_diagrams = os.path.join(pair_dir,"diagrams_all")
            else:
                logging.warning("Results for model: %s not computed in %s",
                                str(outlier_model), str(outlier_model))
        
            self.pairs.append(PairResults(f,path_to_diagrams))
        
class PairResults():
    try:
        all_pairs_metadata = np.loadtxt(os.path.join(os.getcwd(), 'pairs', 'pairmeta.txt'))
    except FileNotFoundError:
        logging.warning("No metadata found! All is set to 0")
        all_pairs_metadata = np.zeros((88, 6))
    
    empty_diagram = dionysus.PersistenceDiagram(0)
    empty_diagram.append((0, 0))
    
    def __init__(self, name, path_to_diagrams, threshold=0):
        if name[-4:] == '.txt':
            name = name[:-4]
        self.name = name
        self.number = int(name[-4:])
        
        self.metadata = self.all_pairs_metadata[self.number-1]
        
        if self.metadata[1] == 1:     # i.e. X --> Y
            self.causality_true = 1  
        else:                         # i.e. Y --> X
            self.causality_true = -1  
        self.weight = self.metadata[5]
        
        self.x_range = range(int(self.metadata[2] - self.metadata[1]))
        self.y_range = range(int(self.metadata[4] - self.metadata[3]))
        
        with open(path_to_diagrams, 'r') as diagrams:
            self.persistence_list = json.load(diagrams)
        
        self.persistence_diagrams = self.create_persistence_diagrams(self.persistence_list)
        
        # self. diagrams_list is a list of dictionaries indexed by outliers.
        # dictionaries contain 4 keys:
        # X
        # X_inverted
        # Y
        # Y_inverted
        
        # Content of every value is (again) a list of persistence diagrams 
        # of projections to axes in x_range or y_range respectively.
        
        # hence to acces persistence pairs of inverse filtration along the 2nd axis of X-variable
        # after removing 10 outliers we need something like
        # self.diagrams_list[10]['X_inverted'][2]
        
        self.X_diagrams = [self.persistence_diagrams[i]["X"]
                           for i in range(len(self.persistence_diagrams))]
        self.Y_diagrams = [self.persistence_diagrams[i]["Y"]
                           for i in range(len(self.persistence_diagrams))]
        self.X_inv_diagrams = [self.persistence_diagrams[i]["X_inverted"] 
                               for i in range(len(self.persistence_diagrams))]
        self.Y_inv_diagrams = [self.persistence_diagrams[i]["Y_inverted"] 
                               for i in range(len(self.persistence_diagrams))]
        
        
        
        
        #----------------------
        #  causality_inferred = 0 (undecided)
        #                   1 (X->Y)
        #                  -1 (Y->X)
        #---------------------
        self.causality_inferred = self.decide_causality()
    
    def create_persistence_diagrams(self, diagram_list):
        """Do not touch"""
        persistence_diagrams = []
        for dictionary in diagram_list:
            dic = {}
            for key, projection in dictionary.items():
                proj = []
                for diagram in projection:
                    p_diagram = dionysus.PersistenceDiagram(0)
                    if not diagram:
                        diagram = [(0,0)]
                    for pair in diagram:
                        p_diagram.append(tuple(pair))
                    proj.append(p_diagram)
                dic[key] = proj
            persistence_diagrams.append(dic)
        return persistence_diagrams
                
        
    def compute_score(self, list_of_persistence_diags, p=0):
        """
        Computes score for a given list of diagrams
        input: persistence_diags_of_projections: one entry of self.{X,Y}_{inv}_diagrams
        return: list of scores
        """
        scores = []
        
        for diagram in list_of_persistence_diags:
            scores.append(self.distance(diagram, p))
        result = max(scores)
                          
        return result
                
    def compute_score_stability(self, persistence_diagrams, p=0):
        """
        input: all the diagrams of projections on a single axis (indexed by outliers)
        return: list of scores for each outlier_removal
        """
        stability = []
        for i in persistence_diagrams:
            stability.append(self.compute_score(i))
        return stability
    
    def decide_causality(self, threshold=0):
        self.X_dist = np.array(self.compute_score_stability(self.X_diagrams))
        self.X_inv_dist = np.array(self.compute_score_stability(self.X_inv_diagrams))

        self.Y_dist = np.array(self.compute_score_stability(self.Y_diagrams))
        self.Y_inv_dist = np.array(self.compute_score_stability(self.Y_inv_diagrams))
                          
        weighting = np.ones(self.X_dist.shape)
                          
        X_integral = np.dot(np.maximum(self.X_dist, self.X_inv_dist), weighting)
        Y_integral = np.dot(np.maximum(self.Y_dist, self.Y_inv_dist), weighting)
        
        self.confidence = np.abs(X_integral - Y_integral)/self.X_dist.shape[0]
                          
        if self.confidence <= threshold:
            causality = 0
        else:
            causality = int((Y_integral - X_integral)/np.abs(X_integral - Y_integral))
        
        return causality
    
    def distance(self, persistence_diagram, p=0):
        """Returns p-th Wasserstein distance between the filtration's 0-diagram
        and the empty diagram.
        If p=0 then the bottleneck is returned.

        TODO: higher dimensions"""

        if p > 0:
            return dionysus.wasserstein_distance(
                persistence_diagram, self.empty_diagram, p)
        else:
            return dionysus.bottleneck_distance(
                persistence_diagram, self.empty_diagram)
    

In [54]:
class AnalysisOfResults:
        
    def __init__(self, prefix='test', threshold=0, outlier_model='knn'):
        
        pattern = re.compile('pair[0-9]{4,}$')
        prefix_path = os.path.join(os.getcwd(),prefix)
        dir_list = sorted([x for x in os.listdir(prefix_path) if pattern.match(x)])
        self.pairs = []
        
        for directory in dir_list:
            pair_dir = os.path.join(os.getcwd(),prefix,directory)
            if outlier_model == 'knn':
                path_to_diagrams = os.path.join(pair_dir,"diagrams_knn")
            elif outlier_model == 'all':
                path_to_diagrams = os.path.join(pair_dir,"diagrams_all")
            else:
                logging.warning("Results for model: %s not computed in %s",
                                str(outlier_model), str(outlier_model))
        
            self.pairs.append(PairResults(directory,path_to_diagrams))
        
        self.pairs.sort(key=lambda x: x.confidence, reverse=True)
        self.pairs_decided = [x for x in self.pairs if x.causality_inferred != 0]
        print(len(self.pairs_decided))
        self.decisions_vector = [x.causality_inferred for x in self.pairs_decided]

        self.decisions_got_right = [x.causality_inferred == x.causality_true 
                                    for x in self.pairs_decided]
        self.weighted_decisions = [x.weight*(x.causality_inferred == x.causality_true)
                                       for x in self.pairs_decided]
        
        self.all_weights = [x.weight for x in self.pairs_decided]
#         self.weighted_efficiency = sum(self.weighted_decisions)/sum(self.all_weights)
        
        #auxilary helper functions
        
#         self.pair_confidences = [[result.number, result.confidence] 
#                           for result in self.results if result.right != -1]
#         self.correct_pair_confidences = [[result.number, result.confidence] 
#                                   for result in self.results if result.right == 1]
#         self.wrong_pair_confidences = [[result.number, result.confidence] 
#                                   for result in self.results if result.right == 0]
#         self.undecided_pair_confidences = [[result.number, result.confidence] 
#                                   for result in self.results if result.right == -1]

    def weighted_efficiency(self,i):
        return sum(self.weighted_decisions[:i])/sum(self.all_weights[:i])
        
    def plot(self, threshold=0):       
        max = len([x for x in self.pairs if 
                   x.confidence >= threshold and 
                   x.causality_inferred != 0])
        to_plot = [self.weighted_efficiency(i) for i in range(1,len(self.decisions_vector)+1)]
        plt.plot(to_plot, color = 'black', alpha = 0.2)
        plt.plot(to_plot[:max], alpha = 0.6)
        plt.ylim(0,1.05)
        print(to_plot[max-1])

In [55]:
X = AnalysisOfResults(prefix, 0.05, outlier_model='knn')

[1.1096990298865399, 1.0016753406120271, 0.87706805158650991, 0.86558173055628118, 0.79101814547822558, 0.64542785928121871, 0.47876490945903938, 0.40375784460263736, 0.40294186888402139, 0.39689168639336975, 0.38629058296200208, 0.35884705128816063, 0.29251705893368646, 0.27439131182974963, 0.26434532625558627, 0.25447271129507165, 0.24900835215433781, 0.24106925854338318, 0.23899698339211947, 0.2279285952824773, 0.22059915126616389, 0.21821995188356669, 0.20127229286336781, 0.1960273352971334, 0.19491294046740876, 0.18474754959872453, 0.17616062284673464, 0.16883358056597833, 0.16820023973451931, 0.16136452367190182, 0.16011013662029885, 0.14425469161772675, 0.13717248949236543, 0.13671323690422232, 0.13529331965478972, 0.134353153362016, 0.13395011064378848, 0.13349963856578825, 0.13084993162571659, 0.13033999521189366, 0.12444943641589923, 0.11589323956499888, 0.11488537600592369, 0.11186694129827361, 0.10530522960359655, 0.097466860672734565, 0.094412671248477853, 0.08913111234359

In [58]:
print(len([p.confidence for p in X.pairs]))

83


In [ ]:
from IPython.html.widgets import widget, interact, interactive, fixed, FloatSlider, Dropdown
        
@interact(threshold = (0,1,0.01))
def dummy(threshold):
    fig = plt.figure(figsize=(12,12))
    fig.set_size_inches(13,6) 
#     Y = AnalysisOfResults(files, prefix, threshold, outlier_model='all')
#     Y.plot(threshold)
    X = AnalysisOfResults(prefix, threshold, outlier_model='knn')
    X.plot(threshold)
    plt.show()